# Winkler score

An alternative way to assess prediction interval coverage for a specific time series is using the Winkler score.  This notebook contains code for the Winkler score as a measure of forecast accuracy of a prediction interval

The winkler score is defined as

$$W_{\alpha,t} = \begin{cases}
  (u_{\alpha,t} - \ell_{\alpha,t}) + \frac{2}{\alpha} (\ell_{\alpha,t} - y_t) & \text{if } y_t < \ell_{\alpha,t} \\
  (u_{\alpha,t} - \ell_{\alpha,t})   & \text{if }  \ell_{\alpha,t} \le y_t \le u_{\alpha,t} \\
  (u_{\alpha,t} - \ell_{\alpha,t}) + \frac{2}{\alpha} (y_t - u_{\alpha,t}) & \text{if } y_t > u_{\alpha,t}.
  \end{cases}$$
  
  
Where 

* $u_{\alpha, t}$ is the upper prediction interval value for $\alpha$ and horizon $t$
* $l_{\alpha, t}$ is the lower prediction interval value for $\alpha$ and horizon $t$
* $y_t$ is the ground truth observation at horizon $t$

## Interpretation

* A Winkler score is the width of the interval plus a penality proportional to the deviation (above or below the interval) and 2/$\alpha$
* Smaller winkler scores are better!

## Imports

In [1]:
import numpy as np
import pandas as pd
import pytest

from forecast_tools.baseline import Naive1

## Winkler score implementation

The above formula is easily implemented as a simple if-else statement.

In [2]:
def winkler_score(l_interval, u_interval, y_t, alpha):
    '''
    Calculates the Winkler score for an observation in 
    an prediction interval.
    
    Winkler scores penalise intervals that do not capture the 
    observation proportional to 2/alpha

    Params:
    -------
    l_interval: float
        Lower prediction interval
        
    u_interval: float
        Upper prediction interval
        
    y_t: float
        Observed ground truth value
        
    alpha: float
        The prediction interval alpha.  For an 80% pred intervals alpha=0.2
        
    Returns:
    -------
    float
    
    Example usage:
    --------------
    ```python
    >> alpha = 0.2
    >> interval = [744.54, 773.22]
    >> y_t = 741.84
    >> ws = winkler_score(interval, y_t, alpha)
    >> print(round(ws, 2))
    
    56.68
    ```
    '''
    score = u_interval - l_interval
    if y_t < l_interval:
        score += ((2/alpha) * (l_interval - y_t))
    elif y_t > u_interval:
        score += ((2/alpha) * (y_t - u_interval))
        
    return score

In [18]:
def mean_winkler_score(intervals, observations, alpha):
    '''
    Returns the mean winkler score across a set of 
    intervals and also a numpy.ndarray of individual winkler scores
    
    Params:
    -------
    intervals: array-like
        array of prediction intervals
        
    observations: array-like
        array of ground truth observations
        
    alpha: float
        The prediction interval alpha.  For an 80% pred intervals alpha=0.2
        
    Returns:
    -------
    float, numpy.ndarray
    
    '''
    if isinstance(observations, pd.DataFrame):
        observations = observations.to_numpy()
    scores = [winkler_score(l[0], l[1], y_t, alpha) for l, y_t in zip(intervals, observations)]
    scores = np.array(scores)
    return scores.mean(), scores

# Simple Example

In [19]:
def test_winker_score(interval, y_t, alpha, expected):
    '''
    Test the winkler score works as expected.
    '''
    ws = winkler_score(interval[0], interval[1], y_t, alpha)
    return expected == pytest.approx(ws, 0.05)

In [20]:
# example data from Forecasting principals and practice
alpha = 0.2 # i.e. a 80% prediction interval
interval = [744.54, 773.22]
y_t = 741.84
expected = 56.68
test_winker_score(interval, y_t, alpha, expected)

True

# Multiple intervals example

Using the outpatient appointments dataset.

In [7]:
url = 'https://raw.githubusercontent.com/health-data-science-OR/' \
        + 'hpdm097-datasets/master/out_appoints_mth.csv'

In [8]:
appoints = pd.read_csv(url, index_col='date', parse_dates=True, dayfirst=True)
appoints.index.freq = 'MS'

In [9]:
train = appoints[:-6]
test = appoints[-6:]

In [10]:
test.shape

(6, 1)

In [11]:
test.to_numpy()[0]

array([47563])

In [12]:
model = Naive1()
preds, intervals = model.fit_predict(train, 6, return_predict_int=True, alpha=[0.2])

In [13]:
intervals[0]

array([[45713.46342943, 71484.53657057],
       [40376.09942344, 76821.90057656],
       [36280.59597698, 80917.40402302],
       [32827.92685885, 84370.07314115],
       [29786.06430164, 87411.93569836],
       [27036.01034012, 90161.98965988]])

In [14]:
mean_score, scores = mean_winkler_score(intervals[0], test, alpha=0.2)

In [15]:
mean_score

46524.61322318127

In [16]:
scores

array([25771.07314115, 36445.80115312, 44636.80804604, 51542.1462823 ,
       57625.87139673, 63125.97931976])